# Topic 3: Estimation and Loss Functions

## Learning Objectives
- Understand Bayesian point estimation
- Master different loss functions and their optimal estimators
- Compare Bayesian and frequentist interval estimation
- Apply decision theory to parameter estimation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pymc as pm
import arviz as az

plt.style.use('seaborn-v0_8')
np.random.seed(42)

## 1. Bayesian Point Estimation

### Common Loss Functions and Optimal Estimators:

| Loss Function | Formula | Optimal Estimator |
|---------------|---------|------------------|
| Squared Error | $L(\theta, \hat{\theta}) = (\theta - \hat{\theta})^2$ | Posterior Mean |
| Absolute Error | $L(\theta, \hat{\theta}) = |\theta - \hat{\theta}|$ | Posterior Median |
| 0-1 Loss | $L(\theta, \hat{\theta}) = \mathbb{I}(\theta \neq \hat{\theta})$ | Posterior Mode |

### Expected Loss (Risk):
$$R(\hat{\theta}) = E[L(\theta, \hat{\theta})] = \int L(\theta, \hat{\theta}) p(\theta|D) d\theta$$

In [ ]:
# Demonstrate different point estimators
# Data: Beta-Binomial example
alpha_prior, beta_prior = 2, 3
successes, trials = 8, 15

# Posterior
alpha_post = alpha_prior + successes
beta_post = beta_prior + trials - successes
posterior = stats.beta(alpha_post, beta_post)

# Point estimates
posterior_mean = posterior.mean()
posterior_median = posterior.median()
posterior_mode = (alpha_post - 1) / (alpha_post + beta_post - 2)  # For Beta distribution

print(f"Posterior: Beta({alpha_post}, {beta_post})")
print(f"Posterior Mean (squared loss): {posterior_mean:.4f}")
print(f"Posterior Median (absolute loss): {posterior_median:.4f}")
print(f"Posterior Mode (0-1 loss): {posterior_mode:.4f}")
print(f"MLE: {successes/trials:.4f}")

# Visualization
x = np.linspace(0, 1, 1000)
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(x, posterior.pdf(x), 'b-', linewidth=2, label='Posterior')
plt.axvline(posterior_mean, color='red', linestyle='-', label=f'Mean = {posterior_mean:.3f}')
plt.axvline(posterior_median, color='green', linestyle='--', label=f'Median = {posterior_median:.3f}')
plt.axvline(posterior_mode, color='orange', linestyle=':', label=f'Mode = {posterior_mode:.3f}')
plt.axvline(successes/trials, color='black', linestyle='-.', label=f'MLE = {successes/trials:.3f}')
plt.xlabel('θ')
plt.ylabel('Density')
plt.title('Point Estimators')
plt.legend()
plt.grid(True, alpha=0.3)

# Risk comparison
theta_true = 0.6  # Assume true value
estimators = {
    'Mean': posterior_mean,
    'Median': posterior_median,
    'Mode': posterior_mode,
    'MLE': successes/trials
}

squared_errors = {name: (est - theta_true)**2 for name, est in estimators.items()}
absolute_errors = {name: abs(est - theta_true) for name, est in estimators.items()}

plt.subplot(1, 2, 2)
names = list(estimators.keys())
sq_errs = [squared_errors[name] for name in names]
abs_errs = [absolute_errors[name] for name in names]

x_pos = np.arange(len(names))
width = 0.35

plt.bar(x_pos - width/2, sq_errs, width, label='Squared Error', alpha=0.7)
plt.bar(x_pos + width/2, abs_errs, width, label='Absolute Error', alpha=0.7)

plt.xlabel('Estimator')
plt.ylabel('Error')
plt.title(f'Estimation Errors (True θ = {theta_true})')
plt.xticks(x_pos, names)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Credible Intervals vs Confidence Intervals

### Credible Interval (Bayesian):
- **Interpretation**: "There is a 95% probability that θ lies in this interval"
- **Construction**: Find interval [a,b] such that $P(a ≤ θ ≤ b|D) = 0.95$

### Confidence Interval (Frequentist):
- **Interpretation**: "If we repeat this procedure many times, 95% of intervals will contain the true θ"
- **Construction**: Based on sampling distribution of estimator

In [ ]:
# Compare credible and confidence intervals
# Simulation study

def simulate_intervals(true_p, n_trials, n_simulations=1000):
    """
    Simulate Bayesian credible intervals and frequentist confidence intervals
    """
    # Storage
    bayesian_covers = []
    frequentist_covers = []
    bayesian_widths = []
    frequentist_widths = []
    
    for _ in range(n_simulations):
        # Generate data
        successes = np.random.binomial(n_trials, true_p)
        
        # Bayesian credible interval (uniform prior)
        posterior = stats.beta(1 + successes, 1 + n_trials - successes)
        bay_ci = posterior.ppf([0.025, 0.975])
        
        # Frequentist confidence interval (Wald)
        p_hat = successes / n_trials
        se = np.sqrt(p_hat * (1 - p_hat) / n_trials)
        freq_ci = [p_hat - 1.96 * se, p_hat + 1.96 * se]
        freq_ci = [max(0, freq_ci[0]), min(1, freq_ci[1])]  # Truncate to [0,1]
        
        # Check coverage
        bayesian_covers.append(bay_ci[0] <= true_p <= bay_ci[1])
        frequentist_covers.append(freq_ci[0] <= true_p <= freq_ci[1])
        
        # Calculate widths
        bayesian_widths.append(bay_ci[1] - bay_ci[0])
        frequentist_widths.append(freq_ci[1] - freq_ci[0])
    
    return {
        'bayesian_coverage': np.mean(bayesian_covers),
        'frequentist_coverage': np.mean(frequentist_covers),
        'bayesian_width': np.mean(bayesian_widths),
        'frequentist_width': np.mean(frequentist_widths)
    }

# Run simulation
true_p = 0.3
n_trials = 50
results = simulate_intervals(true_p, n_trials)

print(f"Simulation Results (true p = {true_p}, n = {n_trials}):")
print(f"Bayesian 95% credible interval coverage: {results['bayesian_coverage']:.3f}")
print(f"Frequentist 95% confidence interval coverage: {results['frequentist_coverage']:.3f}")
print(f"Average Bayesian interval width: {results['bayesian_width']:.3f}")
print(f"Average Frequentist interval width: {results['frequentist_width']:.3f}")

# Visualize single example
np.random.seed(42)
successes = np.random.binomial(n_trials, true_p)
p_hat = successes / n_trials

# Bayesian
posterior = stats.beta(1 + successes, 1 + n_trials - successes)
bay_ci = posterior.ppf([0.025, 0.975])

# Frequentist
se = np.sqrt(p_hat * (1 - p_hat) / n_trials)
freq_ci = [p_hat - 1.96 * se, p_hat + 1.96 * se]

plt.figure(figsize=(12, 4))

# Posterior distribution
plt.subplot(1, 2, 1)
x = np.linspace(0, 0.8, 1000)
plt.plot(x, posterior.pdf(x), 'b-', linewidth=2, label='Posterior')
plt.axvspan(bay_ci[0], bay_ci[1], alpha=0.3, color='blue', label='95% Credible Interval')
plt.axvline(true_p, color='red', linestyle='--', label=f'True p = {true_p}')
plt.axvline(p_hat, color='green', linestyle=':', label=f'Observed p̂ = {p_hat:.2f}')
plt.xlabel('p')
plt.ylabel('Density')
plt.title('Bayesian Credible Interval')
plt.legend()
plt.grid(True, alpha=0.3)

# Comparison
plt.subplot(1, 2, 2)
plt.errorbar([1], [p_hat], yerr=[[p_hat - bay_ci[0]], [bay_ci[1] - p_hat]], 
            fmt='bo', capsize=5, capthick=2, label='Bayesian 95% CI')
plt.errorbar([2], [p_hat], yerr=[[p_hat - freq_ci[0]], [freq_ci[1] - p_hat]], 
            fmt='ro', capsize=5, capthick=2, label='Frequentist 95% CI')
plt.axhline(true_p, color='black', linestyle='--', label=f'True p = {true_p}')
plt.xlim(0.5, 2.5)
plt.xticks([1, 2], ['Bayesian', 'Frequentist'])
plt.ylabel('p')
plt.title('Interval Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nSingle Example (observed {successes}/{n_trials} successes):")
print(f"Bayesian 95% CI: [{bay_ci[0]:.3f}, {bay_ci[1]:.3f}] (width: {bay_ci[1]-bay_ci[0]:.3f})")
print(f"Frequentist 95% CI: [{freq_ci[0]:.3f}, {freq_ci[1]:.3f}] (width: {freq_ci[1]-freq_ci[0]:.3f})")

## 3. Decision Theory Application

### Example: Quality Control Decision
- **Action A₁**: Accept batch (cost = 0 if good, cost = 100 if bad)
- **Action A₂**: Reject batch (cost = 10 always)

**Decision Rule**: Choose action that minimizes expected loss

In [ ]:
# Quality control decision problem
# Defect rate θ, observe x defects in n items

def quality_control_decision(x_defects, n_items, alpha_prior=1, beta_prior=1, 
                           cost_accept_good=0, cost_accept_bad=100, cost_reject=10,
                           threshold_bad=0.05):
    """
    Make quality control decision using Bayesian decision theory
    """
    # Posterior
    alpha_post = alpha_prior + x_defects
    beta_post = beta_prior + n_items - x_defects
    posterior = stats.beta(alpha_post, beta_post)
    
    # Probability batch is "bad" (defect rate > threshold)
    prob_bad = 1 - posterior.cdf(threshold_bad)
    prob_good = 1 - prob_bad
    
    # Expected costs
    expected_cost_accept = (prob_good * cost_accept_good + 
                           prob_bad * cost_accept_bad)
    expected_cost_reject = cost_reject
    
    # Decision
    if expected_cost_accept < expected_cost_reject:
        decision = "Accept"
        min_cost = expected_cost_accept
    else:
        decision = "Reject"
        min_cost = expected_cost_reject
    
    return {
        'decision': decision,
        'prob_bad': prob_bad,
        'expected_cost_accept': expected_cost_accept,
        'expected_cost_reject': expected_cost_reject,
        'min_expected_cost': min_cost,
        'posterior_mean': posterior.mean()
    }

# Example scenarios
scenarios = [
    (2, 100, "Low defects"),
    (5, 100, "Medium defects"),
    (8, 100, "High defects"),
    (12, 100, "Very high defects")
]

print("Quality Control Decision Analysis:")
print("Costs: Accept good batch = 0, Accept bad batch = 100, Reject = 10")
print("Bad batch threshold: defect rate > 5%")
print("\nScenario\t\tDefects\tP(Bad)\tDecision\tExpected Cost")
print("-" * 65)

results = []
for x_defects, n_items, description in scenarios:
    result = quality_control_decision(x_defects, n_items)
    results.append(result)
    
    print(f"{description:<15}\t{x_defects}/{n_items}\t{result['prob_bad']:.3f}\t{result['decision']}\t\t{result['min_expected_cost']:.1f}")

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

for i, ((x_defects, n_items, description), result) in enumerate(zip(scenarios, results)):
    # Posterior distribution
    alpha_post = 1 + x_defects
    beta_post = 1 + n_items - x_defects
    posterior = stats.beta(alpha_post, beta_post)
    
    x = np.linspace(0, 0.2, 1000)
    axes[i].plot(x, posterior.pdf(x), 'b-', linewidth=2, label='Posterior')
    axes[i].axvline(0.05, color='red', linestyle='--', label='Threshold (5%)')
    
    # Shade "bad" region
    x_bad = x[x >= 0.05]
    y_bad = posterior.pdf(x_bad)
    axes[i].fill_between(x_bad, y_bad, alpha=0.3, color='red', 
                        label=f'P(Bad) = {result["prob_bad"]:.3f}')
    
    axes[i].set_title(f'{description}\nDecision: {result["decision"]} (Cost: {result["min_expected_cost"]:.1f})')
    axes[i].set_xlabel('Defect Rate')
    axes[i].set_ylabel('Density')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Decision boundary analysis
x_range = range(0, 15)
decisions = []
costs = []

for x in x_range:
    result = quality_control_decision(x, 100)
    decisions.append(1 if result['decision'] == 'Accept' else 0)
    costs.append(result['min_expected_cost'])

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
colors = ['red' if d == 0 else 'green' for d in decisions]
plt.bar(x_range, [1]*len(x_range), color=colors, alpha=0.7)
plt.xlabel('Number of Defects (out of 100)')
plt.ylabel('Decision')
plt.title('Decision Boundary')
plt.yticks([0, 1], ['Reject', 'Accept'])
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(x_range, costs, 'bo-', linewidth=2, markersize=6)
plt.xlabel('Number of Defects (out of 100)')
plt.ylabel('Minimum Expected Cost')
plt.title('Expected Cost vs Defects')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Key Takeaways

### Point Estimation:
- **Choice of loss function** determines optimal estimator
- **Squared loss** → posterior mean (most common)
- **Absolute loss** → posterior median (robust)
- **0-1 loss** → posterior mode (MAP estimate)

### Interval Estimation:
- **Credible intervals**: Direct probability statements about parameters
- **Confidence intervals**: Frequency properties of the procedure
- **Bayesian interpretation** is more intuitive for decision making

### Decision Theory:
- **Formal framework** for incorporating costs and utilities
- **Expected loss minimization** provides optimal decisions
- **Sensitivity analysis** important for cost assumptions

## Next: Topic 4 - Hypothesis Testing and Model Comparison